In [1]:
import os

%pwd

'/Users/mitpatel/Downloads/Code/GoJo-Rika/MLOPS/Text-Summarizer/research'

In [2]:
os.chdir("../")

In [12]:
%pwd

'/Users/mitpatel/Downloads/Code/GoJo-Rika/MLOPS/Text-Summarizer'

In [ ]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    per_device_eval_batch_size: int
    weight_decay: float
    logging_steps: int
    eval_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int


In [ ]:
from src.text_summarizer.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.text_summarizer.utils.common import create_directories, read_yaml


In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath: str = CONFIG_FILE_PATH,
        params_filepath: str = PARAMS_FILE_PATH,
    ) -> None:
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt,
            num_train_epochs=params.num_train_epochs,
            warmup_steps=params.warmup_steps,
            per_device_train_batch_size=params.per_device_train_batch_size,
            per_device_eval_batch_size=params.per_device_eval_batch_size,
            weight_decay=params.weight_decay,
            logging_steps=params.logging_steps,
            eval_strategy=params.eval_strategy,
            eval_steps=params.eval_steps,
            save_steps=params.save_steps,
            gradient_accumulation_steps=params.gradient_accumulation_steps,
        )
        return model_trainer_config


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
import torch
from datasets import load_from_disk

In [ ]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig) -> None:
        self.config = config

    def train(self):

        if torch.backends.mps.is_available(): # Detect device (MPS for Apple Silicon)
            device = "mps"
        elif torch.cuda.is_available(): # Detect NVIDIA GPU (cuda for GPU)
            device = "cuda"
        else:
            device = "cpu"
        # logger.info(f"Using device: {device}")
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        # Load preprocessed dataset
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=self.params.num_train_epochs,
            warmup_steps=self.params.warmup_steps,
            per_device_train_batch_size=self.params.per_device_train_batch_size,
            per_device_eval_batch_size=self.params.per_device_eval_batch_size,
            weight_decay=self.params.weight_decay,
            logging_steps=self.params.logging_steps,
            eval_strategy=self.params.eval_strategy,
            eval_steps=self.params.eval_steps,
            save_steps=self.params.save_steps,
            gradient_accumulation_steps=self.params.gradient_accumulation_steps,
        )
        trainer = Trainer(
            model=model_pegasus,
            args=trainer_args,
            tokenizer=tokenizer,
            data_collator=seq2seq_data_collator,
            train_dataset=dataset_samsum_pt["test"],
            eval_dataset=dataset_samsum_pt["validation"],
        )

        trainer.train()

        ## Save model and tokenizer
        model_pegasus.save_pretrained(Path(self.config.root_dir) / "pegasus-samsum-model")
        tokenizer.save_pretrained(Path(self.config.root_dir) / "tokenizer")


In [ ]:
import gc
import torch

gc.collect()
if torch.backends.mps.is_available():
    torch.mps.empty_cache()


In [ ]:
config = ConfigurationManager()
model_trainer_config = config.get_model_trainer_config()
model_trainer = ModelTrainer(config=model_trainer_config)
model_trainer.train()